In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/my_data_DS317/train_update.xlsx')
df.drop(['gioitinh'], axis=1)
df.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,39B39397XPvAibaEXe/vGZ1xWLvpK7wMZlgBlCBs,19,0,75.198135,30.0,9.37,9.37,24,9.37,9.34,9.40,9.370000,9.370000,9.370000,9.370000,9.370000
1,3AB392AFXPvAibaEXe/NRwqjPAC0MKjf0QSiSLjM,17,0,85.000000,26.5,7.07,7.07,139,7.00,6.74,6.25,6.230000,7.150000,6.920000,6.970000,7.600000
2,BEC54CF0XPvAibaEXe/qZKhoIW7F1rbPON7lHJUG,19,0,82.000000,30.0,6.54,6.54,18,6.54,6.54,6.54,6.540000,6.540000,6.540000,6.540000,6.540000
3,43386D57XPvAibaEXe/nw3DhNQnHfzJZvI0o00SJ,20,0,75.655535,22.0,1.83,1.83,4,3.56,0.00,1.78,1.780000,1.780000,1.780000,1.780000,1.780000
4,8F1A725FXPvAibaEXe88DMpF1KL2K3HyZfgFjAif,19,0,100.000000,30.0,9.31,9.31,64,9.21,9.32,9.41,9.313333,9.313333,9.313333,9.313333,9.313333


In [ ]:
df_test = pd.read_excel('/content/drive/MyDrive/my_data_DS317/test_update.xlsx')
df_test.drop(['gioitinh'], axis=1)
df_test.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,14EEA146XPvAibaEXe/2sgPA+OKAks3AV3zsZSFH,22,0,82.105913,26.762789,8.77,8.77,123,8.61,8.79,9.01,9.04,8.63,8.620,8.530,8.747143
1,4A2628C7XPvAibaEXe9urngX21O5XaWExeziRFls,19,0,82.105913,30.000000,8.42,8.42,130,9.11,8.64,7.78,8.60,8.45,8.300,9.300,8.597143
2,44634117XPvAibaEXe+WdrMfsV/1fSPAGTu3hQ5U,22,0,82.105913,22.350000,7.73,7.73,137,7.83,7.46,7.53,7.67,7.89,8.030,7.750,7.737143
3,8A7BA5ACXPvAibaEXe8FNnc+c+WuJHdW9KyIp7Nl,19,0,100.000000,30.000000,6.52,6.52,83,7.63,7.21,7.70,7.05,1.67,6.252,6.252,6.252000
4,A3A77666XPvAibaEXe/4WyTnp6RTwi1hkjCNtuCX,20,0,75.655535,24.250000,6.50,6.50,148,6.02,6.75,6.79,6.26,5.21,5.580,6.450,6.690000


In [ ]:
# df = df.drop(['khoa', 'hedt', 'khoahoc', 'diemtbhk_13', 'diemtbhk_14', 'diemtbhk_15', 'diemtbhk_16' , 'diemtbhk_17', 'diemtbhk_18', 'diemtbhk_19'], axis=1)
# df['gioitinh'] = df['gioitinh'].fillna('Nam')
# df['gioitinh'] = df['gioitinh'].replace({'Nam': 0, 'Nữ': 1})
# mean_nam_sinh = int(df['namsinh'].mean(skipna=True))
# df['namsinh'] = df['namsinh'].fillna(mean_nam_sinh)
# df['namsinh'] = df['namsinh'] - df['namsinh'].min()
# #df = df.fillna(0)
# df.head()

In [ ]:

# Hàm xây dựng mô hình Phased LSTM
def build_phased_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(50, activation='tanh', recurrent_activation='sigmoid'),
        Dense(1)  # Đầu ra là một giá trị
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Độ đo
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

# Cột ban đầu cho X
initial_columns = ['namsinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns)+2, len(df.columns) - 1):  # Bắt đầu từ cột tiếp theo sau `initial_columns`
    if i +2 >= len(df.columns):
        break
    target_column = df.columns[i+2]  # Cột y hiện tại
    print(target_column)

    # Xác định X và y
    X_train = df[current_X_columns]
    y_train = df[target_column]

    X_test = df_test[current_X_columns]
    y_test = df_test[target_column]

    # Chuẩn hóa dữ liệu cho Phased LSTM
    X_train_lstm = np.expand_dims(X_train.to_numpy(), axis=-1)  # Thêm chiều
    X_test_lstm = np.expand_dims(X_test.to_numpy(), axis=-1)

    # Xây dựng và huấn luyện mô hình Phased LSTM
    lstm_model = build_phased_lstm(input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
    lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=1)

    # Dự đoán và tính toán độ đo
    y_pred = lstm_model.predict(X_test_lstm).flatten()
    metrics = calculate_metrics(y_test, y_pred)
    metrics["Target Column"] = target_column
    results.append(metrics)

    # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)

# Hiển thị kết quả
print(results_df)

# Lưu kết quả ra file CSV
results_df.to_csv("phased_lstm_results.csv", index=False)


diemtbhk_2
Epoch 1/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 9.9794 - mae: 2.3410 
Epoch 2/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.8654 - mae: 0.6611
Epoch 3/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.8760 - mae: 0.6415
Epoch 4/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.8354 - mae: 0.6214
Epoch 5/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.7741 - mae: 0.5999
Epoch 6/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7641 - mae: 0.5948
Epoch 7/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.8037 - mae: 0.6058
Epoch 8/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.7257 - mae: 0.5820
Epoch 9/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7862 - mae: 0.5988
Epoch 10/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7700 - mae: 0.5892
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
diemtbhk_3
Epoch 1/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 10.5421 - mae: 2.3589
Epoch 2/10
361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 5m

In [ ]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

0.8067215849150966
0.8641419913158834
0.9133574509808516
0.5158518241423101
